# Robotic Systems II (ECE-DK904)

## Electrical and Computer Engineering Department, University of Patras, Greece

**Instructor:** Konstantinos Chatzilygeroudis (costashatz@upatras.gr)

## Lab 6

In this lab, we are going to implement the basic math for $SO(3)$ and $SE(3)$ spaces.

First, let's get some basic math.

In [ ]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

np.set_printoptions(precision=2, suppress=True)

# Rotations around principal axes
def RotX(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[1, 1] = ct
    R[1, 2] = -st
    R[2, 1] = st
    R[2, 2] = ct
    return R

def RotY(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 2] = st
    R[2, 0] = -st
    R[2, 2] = ct
    return R

def RotZ(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 1] = -st
    R[1, 0] = st
    R[1, 1] = ct
    return R

In [ ]:
# hat, unhat operations
def hat(vec):
    v = vec.reshape((3,))
    return np.array([
        [0., -v[2], v[1]],
        [v[2], 0., -v[0]],
        [-v[1], v[0], 0.]
    ])

def unhat(mat):
    return np.array([[mat[2, 1], mat[0, 2], mat[1, 0]]]).T

# exp for SO(3)
def exp_rotation(p):
    phi = p.reshape((3, 1))
    theta = np.linalg.norm(phi)
    if theta < 1e-12:
        return np.eye(3, 3)
    a = phi / theta
    return np.eye(3) * np.cos(theta) + (1. - np.cos(theta)) * a @ a.T + np.sin(theta) * hat(a)

# log for SO(3)
def log_rotation(R):
    theta = np.arccos(max(-1., min(1., (np.trace(R) - 1.) / 2.)))

    if np.isclose(theta, 0.):
        return np.zeros((3, 1))
    elif np.isclose(theta, np.pi):
        r00 = R[0, 0]
        r11 = R[1, 1]
        r22 = R[2, 2]

        r02 = R[0, 2]
        r12 = R[1, 2]

        r01 = R[0, 1]
        r21 = R[2, 1]

        r10 = R[1, 0]
        r20 = R[2, 0]

        if not np.isclose(r22, -1.):
            multiplier = theta / np.sqrt(2. * (1. + r22))
            return multiplier * np.array([[r02, r12, 1. + r22]]).T
        elif not np.isclose(r11, -1.):
            multiplier = theta / np.sqrt(2. * (1. + r11))
            return multiplier * np.array([[r01, 1. + r11, r21]]).T
        elif not np.isclose(r00, -1.):
            multiplier = theta / np.sqrt(2. * (1. + r00))
            return multiplier * np.array([[1. + r00, r10, r20]]).T
        else:
            print("ERROR: This should never happen!")
            exit(1)

    mat = R - R.T
    r = unhat(mat)

    return theta / (2. * np.sin(theta)) * r

Now let's implement the following functions:

$\boldsymbol{J}_l(\boldsymbol{\phi}) = \boldsymbol{I} + \frac{1-\cos\phi}{\phi^2}\boldsymbol{\phi}^\land + \frac{\phi-\sin\phi}{\phi^3}\boldsymbol{\phi}^\land\boldsymbol{\phi}^\land$

$\boldsymbol{J}_l^{-1}(\boldsymbol{\phi}) = \boldsymbol{I} - \frac{1}{2}\boldsymbol{\phi}^\land + (\frac{1}{\phi^2}-\frac{1+\cos\phi}{2\phi\sin\phi})\boldsymbol{\phi}^\land\boldsymbol{\phi}^\land$

We have $\boldsymbol{\phi} = \phi\boldsymbol{r}$.

In [ ]:
def J_l(q, epsilon = 1e-8):
    ### TO-DO: Implement the $J_l$ function. You should return a numpy array with dimensions 3x3.
    phi = np.linalg.norm(q)
    phi_bold = q
    I = np.eye(3)
    
    if(phi < epsilon):
        J = I
    else:
        J = I + ((1 - np.cos(phi)) / (phi ** 2)) * hat(phi_bold) + ((phi - np.sin(phi)) / (phi ** 3)) * (hat(phi_bold) @ hat(phi_bold))
    
    return J
    ### END of TO-DO

def J_l_inv(q, epsilon = 1e-8):
    ### TO-DO: Implement the $J_l^{-1}$ function. You should return a numpy array with dimensions 3x3.
    phi = np.linalg.norm(q)
    I = np.eye(3)
    
    if(phi < epsilon):
        J = I
    else:
        J = I - 0.5 * hat(q) + ((1 / (phi ** 2)) - ((1 + np.cos(phi)) / (2 * phi * np.sin(phi)))) * (hat(q) @ hat(q))

    return J
    ### END of TO-DO

In [ ]:
assert(np.isclose(J_l(np.zeros((3, 1))), np.eye(3)).all())
assert(np.isclose(J_l_inv(np.zeros((3, 1))), np.eye(3)).all())

phi = np.array([[1., 0., 0.]]).T
J_l_phi = np.array([[1., 0., 0.], [0., 0.84, -0.46], [0., 0.46, 0.84]])
J_l_inv_phi = np.array([[1., 0., 0.], [0., 0.92, 0.5], [0., -0.5, 0.92]])
assert(np.isclose(J_l(phi), J_l_phi, rtol=1e-2).all())
assert(np.isclose(J_l_inv(phi), J_l_inv_phi, rtol=1e-2).all())

phi = np.array([[0., 1., 0.]]).T
J_l_phi = np.array([[0.84, 0., 0.46], [0., 1., 0.], [-0.46, 0., 0.84]])
J_l_inv_phi = np.array([[0.92, 0., -0.5], [0., 1., 0.], [0.5, 0., 0.92]])
assert(np.isclose(J_l(phi), J_l_phi, rtol=1e-2).all())
assert(np.isclose(J_l_inv(phi), J_l_inv_phi, rtol=1e-2).all())

for _ in range(50):
    phi = np.random.rand(3, 1) * 10. - 5.
    assert(np.isclose(np.linalg.inv(J_l(phi)), J_l_inv(phi)).all())
    assert(np.isclose(J_l(phi) @ J_l_inv(phi), np.eye(3)).all())
    isZero = np.linalg.norm(phi) < 1e-8
    if not isZero:
        assert(not np.isclose(J_l(phi), np.eye(3)).all())
        assert(not np.isclose(J_l_inv(phi), np.eye(3)).all())

Now, let's implement the $\exp$ and $\log$ operations for $SE(3)$:

$\boldsymbol{\xi} = \begin{bmatrix}\boldsymbol{\phi}\\\boldsymbol{\rho}\end{bmatrix}\in\mathbb{R}^6$

$\exp(\boldsymbol{\xi}^\land) = \boldsymbol{T} = \begin{bmatrix}\exp(\boldsymbol{\phi}^\land) & \boldsymbol{J}_l(\boldsymbol{\phi})\boldsymbol{\rho}\\\boldsymbol{0} & 1\end{bmatrix}$

$\log(\boldsymbol{T})^\lor = \boldsymbol{\xi} = \begin{bmatrix}\log(\boldsymbol{R})^\lor\\\boldsymbol{J}_l^{-1}(\boldsymbol{\phi})\boldsymbol{t}\end{bmatrix}$

where $\boldsymbol{\xi}^\land = \begin{bmatrix}\boldsymbol{\phi}^\land & \boldsymbol{\rho}\\\boldsymbol{0} & 0\end{bmatrix}$

In [ ]:
def skew_pose(xi):
    ### TO-DO: Implement the skew operator for SE(3): ξ^. You should return a numpy array with dimensions 4x4.
    phi = np.array([xi[0], xi[1], xi[2]])
    rho = np.array([xi[3], xi[4], xi[5]])
    
    arr = np.block([[      hat(phi), rho],
                    [np.zeros((3,)),   0]])
    return arr
    ### END of TO-DO

def unskew_pose(T):
    ### TO-DO: Implement the un-skew operator for SE(3). You should return a numpy array with dimensions 6x1.
    phi = unhat(T[:3, :3])
    rho = np.array([T[:3, 3]]).reshape(3,1)
    vec = np.block([[phi], [rho]])
    
    return vec
    ### END of TO-DO

def exp_pose(xi):
    ### TO-DO: Implement the exp operator for SE(3). You should return a numpy array with dimensions 4x4.
    phi = np.array([xi[0], xi[1], xi[2]])
    rho = np.array([xi[3], xi[4], xi[5]])
    
    arr = np.block([[exp_rotation(phi), J_l(phi) @ rho],
                    [   np.zeros((3,)),              1]])
    
    return arr
    ### END of TO-DO

def log_pose(T):
    ### TO-DO: Implement the log operator for SE(3). You should return a numpy array with dimensions 6x1.
    R = T[:3, :3]
    t = np.array([T[:3, 3]]).reshape(3,1)
    phi = log_rotation(R)

    vec = np.block([[phi], [J_l_inv(phi) @ t]])
    
    return vec
    ### END of TO-DO

In [ ]:
from scipy.linalg import expm
from scipy.linalg import logm

xi = np.array([[1., -3., 0.2, 2., 3., 1.]]).T
T = np.eye(4)
T[:3, :3] = RotX(np.pi / 3.)
T[:3, 3:] = np.array([[3., 2., -1.]]).T

assert(np.isclose(exp_pose(xi), expm(skew_pose(xi))).all())
assert(np.isclose(unskew_pose(logm(T)), log_pose(T)).all())

for _ in range(50):
    xi = np.random.rand(6, 1) * 10. - 5.
    assert(np.isclose(exp_pose(xi), expm(skew_pose(xi))).all())

    T = np.eye(4)
    T[:3, :3] = RotX(np.random.rand() * 2. * np.pi - np.pi) @ RotY(np.random.rand() * 2. * np.pi - np.pi) @ RotZ(np.random.rand() * 2. * np.pi - np.pi)
    T[:3, 3:] = np.random.rand(3, 1) * 10. - 5.
    assert(np.isclose(unskew_pose(logm(T)), log_pose(T)).all())